In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy.stats import norm, skew, boxcox_normmax
import warnings
from sklearn import decomposition
from scipy.special import boxcox1p
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
train = pd.read_csv('../input/moviealldata/movietrain5.csv')
test = pd.read_csv('../input/moviealldata/movietest5.csv')

In [ ]:
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

train_ID = train.pop('id')
test_ID = test.pop('id')

In [ ]:
train["Total score"] = np.log1p(train["Total score"])

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train['Total score'].values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['Total score'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

In [ ]:
#one hot 方法
def generate_one_hot(df,sentence_col):
   temp=[]
   temp=np.array(pd.get_dummies(df[sentence_col]))
   #print(temp)
   for i in range(temp[0].size):
          df[sentence_col+'_{}'.format(i)]=temp[:,i]
    
   return df

generate_one_hot(all_data,"Motion Picture Rating(MPAA)")
all_data=all_data.drop("Motion Picture Rating(MPAA)",axis=1)

#generate_one_hot(X_train,"countries of origina")
all_data=all_data.drop("countries of origina",axis=1)

#generate_one_hot(X_train,"language")
all_data=all_data.drop("language",axis=1)

generate_one_hot(all_data,"movie series")
all_data=all_data.drop("movie series",axis=1)
all_data=all_data.drop("stars",axis=1)
all_data=all_data.drop("film",axis=1)
all_data=all_data.drop("time",axis=1)
all_data=all_data.drop("Production Companies",axis=1)
all_data=all_data.drop("Script",axis=1)
all_data=all_data.drop("Directors",axis=1)    
all_data=all_data.drop("writers",axis=1)
all_data=all_data.drop("Filming locations",axis=1)

all_data=all_data.drop("Grnres",axis=1)
#train_df=train_df.drop("score",axis=1)
all_data.head()

In [ ]:
#all_data['year']=all_data['year'].fillna('0') 
all_data=all_data.fillna('0') 
#np.isnan(all_data)
#np.where(np.isnan(all_data))
#all_data['year'].isnull()
all_data.head()

In [ ]:

all_data = pd.get_dummies(all_data)
print(all_data.shape)

pca = decomposition.PCA()

train = all_data[:ntrain].values
test = all_data[ntrain:].values

In [ ]:
#from sklearn.impute import SimpleImputer
#my_imputer = SimpleImputer()
#data_with_imputed_values = my_imputer.fit_transform(train)

In [ ]:
# model begin ====================================================================== #

from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb
import lightgbm as lgb

n_folds = 5


def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train)
    mse = np.sqrt(-cross_val_score(model, train, y_train, scoring="neg_mean_squared_error", cv=kf))
    return mse


kfolds = KFold(n_splits=n_folds, shuffle=True, random_state=42)
alph = [0.01, 0.001, 0.0001, 0.0002, 0.0004, 0.0008, 0.002, 0.004, 0.008, 1, 2, 4, 6, 8, 10, 12]
alph2 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
lasso = make_pipeline(RobustScaler(), LassoCV(alphas=alph, cv=kfolds, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNetCV(alphas=alph, l1_ratio=.9, cv=kfolds, random_state=3))
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alph2, cv=kfolds))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)


GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state=5)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=2, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=True,
                             random_state=7, nthread=-1)

model_lgb = lgb.LGBMRegressor(objective='regression', num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin=55, bagging_fraction=0.8,
                              bagging_freq=5, feature_fraction=0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf=6, min_sum_hessian_in_leaf=11)

stacked_averaged_models = StackingCVRegressor(regressors=(ENet, GBoost, KRR),
                                              meta_regressor=lasso,
                                              use_features_in_secondary=True)


In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
# 下一步，更改融合方式

model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_importance = model_xgb.feature_importances_
xgb_out = np.argsort(xgb_importance)

print(rmsle(y_train, xgb_train_pred))
model_lgb.fit(train, y_train)

In [ ]:
booster = model_lgb.booster_
lgb_importance = booster.feature_importance(importance_type='split')
lgb_out = np.argsort(lgb_importance)

lgb_train_pred = model_lgb.predict(train)
print(rmsle(y_train, lgb_train_pred))
print('RMSLE score on train data:')
# lasso.fit(train, y_train)
# lasso_train_pred = lasso.predict(train)
# lasso_pred = np.expm1(lasso.predict(test))

GBoost.fit(train, y_train)
GBoost_train_pred = GBoost.predict(train)
GBT_feature = GBoost.feature_importances_
gbt_out = np.argsort(GBT_feature)
drop_num = 50
lgb_out = lgb_out[:drop_num]
xgb_out = xgb_out[:drop_num]
gbt_out = gbt_out[:drop_num]
# drop_feature = [val for val in lgb_out if (val in xgb_out and val in gbt_out)]
drop_feature = list(set(lgb_out).union(xgb_out).union(gbt_out))
print(drop_feature)
train = np.delete(train, drop_feature, axis=1)
test = np.delete(test, drop_feature, axis=1)

In [ ]:
# ========================================== pred ===================================#
stacked_averaged_models.fit(train, y_train)
stacked_train_pred = stacked_averaged_models.predict(train)
stacked_pred = np.expm1(stacked_averaged_models.predict(test))
print(rmsle(y_train, stacked_train_pred))

model_lgb.fit(train, y_train)
model_xgb.fit(train, y_train)
xgb_pred = np.expm1(model_xgb.predict(test))
# GBoost_pred = np.expm1(GBoost.predict(test))
lgb_pred = np.expm1(model_lgb.predict(test))
print(rmsle(y_train, stacked_train_pred * 0.7 + xgb_train_pred * 0.15 + lgb_train_pred * 0.15))
#ensemble = stacked_pred * 0.7 + xgb_pred * 0.15 + lgb_pred * 0.15
ensemble = (stacked_pred + xgb_pred + lgb_pred)/3
sub_LGB = pd.DataFrame()
sub_LGB['id'] = test_ID
sub_LGB['Total score'] = lgb_pred
sub_XGB = pd.DataFrame()
sub_XGB['id'] = test_ID
sub_XGB['Total score'] = xgb_pred
sub_LASSO = pd.DataFrame()
sub_LASSO['id'] = test_ID
sub_LASSO['Total score'] = stacked_pred
sub_ensemble = pd.DataFrame()
sub_ensemble['id'] = test_ID
sub_ensemble['Total score'] = stacked_pred
#q1 = submission['Total score'].quantile(0.005)
#q2 = submission['Total score'].quantile(0.995)
#submission['Total score'] = submission['Total score'].apply(lambda x: x if x > q1 else x * 0.85)
#submission['Total score'] = submission['Total score'].apply(lambda x: x if x < q2 else x * 1.1)
sub_LGB.to_csv('./sub_LGB.csv', index=False)
sub_XGB.to_csv('./sub_XGB.csv', index=False)
sub_LASSO.to_csv('./sub_LASSO.csv', index=False)
sub_ensemble.to_csv('./sub_ensemble.csv', index=False)

In [ ]:
from sklearn.svm import LinearSVC
#from sklearn.learning_curve import learning_curve
from sklearn.datasets import load_digits # 导入手写数字集
from sklearn.model_selection import learning_curve
#绘制学习曲线，以确定模型的状况
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        train_sizes=np.linspace(.1, 1.0, 5)):
#     """
#     画出data在某模型上的learning curve.
#     参数解释
#     ----------
#     estimator : 你用的分类器。
#     title : 表格的标题。
#     X : 输入的feature，numpy类型
#     y : 输入的target vector
#     ylim : tuple格式的(ymin, ymax), 设定图像中纵坐标的最低点和最高点
#     cv : 做cross-validation的时候，数据分成的份数，其中一份作为cv集，其余n-1份作为training(默认为3份)
#     """
    plt.figure()
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, n_jobs=1, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.legend(loc="best")
    plt.grid("on") 
    if ylim:
        plt.ylim(ylim)
    plt.title(title)
    plt.show()
#少样本的情况情况下绘出学习曲线
plot_learning_curve(LinearSVC(C=10.0), "LinearSVC(C=10.0)",
                    train, y_train, ylim=(0.1, 1.01),
                    train_sizes=np.linspace(.1, 1, 5))